This script covers the "GeoApify" extraction portion of the project. Because this requires running queries through GeoApify, it is better to have it as a separate file to save the results to a CSV, rather than attempting to integrate into the main statistics and visualizations script and rerunning through obtaining the same data each and every time (particularly as APi requests are limited each day)

In [1]:
#import required libraries

import pandas as pd
import numpy as np
import requests
from api_keys import geoapify_key

In [2]:
#Read base dataset that was provided to team

tor_df = pd.read_csv("../Data/tor_data.csv")
van_df= pd.read_csv("../Data/van_data.csv")


In [3]:
#In order to input coordinates into GeoApify, it is easiest to combine latitude and longtitude into one string

tor_df["coordinates"]=tor_df["Latitude"].astype(str)+","+tor_df["Longitude"].astype(str)
tor_df["Distance"]=''
tor_df.head()

,Restaurant Name,Address,Category,Rating,Number of Reviews,Price($),Price(1-5),Latitude,Longitude,coordinates,Distance
0,Pai Northern Thai Kitchen,18 Duncan Street,Thai,4.5,3335,$$,2,43.647840,-79.388720,"43.64784,-79.38872",
1,Richmond Station,1 Richmond Street W,Canadian (New),4.5,869,$$$,3,43.651373,-79.379285,"43.6513729,-79.379285",
2,Mira,420A Wellington Street W,Peruvian,4.5,256,$$,2,43.644170,-79.396390,"43.64417,-79.39639",
3,Rasa,196 Robert Street,Canadian (New),4.5,317,$$,2,43.662775,-79.403968,"43.6627752,-79.4039682",
4,KINKA IZAKAYA ORIGINAL,398 Church Street,Japanese,4.0,1790,$$,2,43.660424,-79.378900,"43.66042406856627,-79.37890046282942",


In [4]:
van_df["coordinates"]=van_df["Latitude"].astype(str)+","+van_df["Longitude"].astype(str)

Set the base_URL for GeoApify, and also include some key parameters. For the routing API, this simply consists of the mode being set to "drive", which gives a middle optimized return of driving route/directions/distance (which we want) from each landmark to a specific restaurant

In [5]:
base_url="https://api.geoapify.com/v1/routing?"
params = {
    "apiKey":geoapify_key,    
    "mode": "drive"
}

For landmarks, it is somewhat subjective but they were chosen to be the CN Tower in Toronto and Canada Place in Vancouver. Finding their co-ordinates via web search yields the following:

In [6]:

tor_landmark="43.642567,-79.387054"
van_landmark="49.288635,-123.111119"

Go through each row in the dataframe, extract coordinates, calculate the driving distance, extrac the driving distance and input into a new column within each dataframe. Export this as a CSV for the next script to use.

In [7]:

for index, row in tor_df.iterrows():
    
    # Get the coordinate from the current DataFrame's row
    coordinates = tor_df.loc[index, "coordinates"]
    # Add the current ethnicity type to the parameters
    params["waypoints"] = coordinates+"|"+tor_landmark
   
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()
    
    try:
        tor_df.loc[index, "Distance"] = response["features"][0]["properties"]["distance"]
    except (KeyError, IndexError):
        tor_df.loc[index, "Distance"] = np.nan
    
tor_df.to_csv("../Data/tor_data_ChrisEdit.csv")
tor_df.head()

,Restaurant Name,Address,Category,Rating,Number of Reviews,Price($),Price(1-5),Latitude,Longitude,coordinates,Distance
0,Pai Northern Thai Kitchen,18 Duncan Street,Thai,4.5,3335,$$,2,43.647840,-79.388720,"43.64784,-79.38872",1485
1,Richmond Station,1 Richmond Street W,Canadian (New),4.5,869,$$$,3,43.651373,-79.379285,"43.6513729,-79.379285",2001
2,Mira,420A Wellington Street W,Peruvian,4.5,256,$$,2,43.644170,-79.396390,"43.64417,-79.39639",1204
3,Rasa,196 Robert Street,Canadian (New),4.5,317,$$,2,43.662775,-79.403968,"43.6627752,-79.4039682",3423
4,KINKA IZAKAYA ORIGINAL,398 Church Street,Japanese,4.0,1790,$$,2,43.660424,-79.378900,"43.66042406856627,-79.37890046282942",3581


In [162]:
for index, row in van_df.iterrows():
    
    # Get the coordinate from the current DataFrame's row
    coordinates = van_df.loc[index, "coordinates"]
    # Add the current ethnicity type to the parameters
    params["waypoints"] = coordinates+"|"+van_landmark
   
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()
    
    try:
        van_df.loc[index, "Distance"] = response["features"][0]["properties"]["distance"]
    except (KeyError, IndexError):
        van_df.loc[index, "Distance"] = np.nan
    
van_df.to_csv("../Data/van_data_ChrisEdit.csv")
van_df.head()

,Restaurant Name,Address,Category,Rating,Number of Reviews,Price($),Price(1-5),Latitude,Longitude,coordinates,Distance
0,The Flying Pig,1168 Hamilton Street,Canadian (New),4.0,1247,$$,2,49.275010,-123.122000,"49.27501,-123.122",2284.0
1,Kosoo,1128 Robson Street,Korean,4.5,69,$$,2,49.284940,-123.125920,"49.28494,-123.12592",1591.0
2,Fat Mao Noodles,938 Helmcken Street,Noodles,4.5,27,NaN,0,49.279902,-123.127246,"49.2799016,-123.1272458",1943.0
3,Miku,200 Granville Street,Japanese,4.5,2107,$$$,3,49.286918,-123.112763,"49.28691753455261,-123.11276314110472",1070.0
4,Kingyo,871 Denman Street,Japanese,4.5,1012,$$,2,49.290610,-123.137140,"49.29061,-123.13714",2544.0
